<a href="https://colab.research.google.com/github/WilliamAshbee/minigan-catalyst/blob/master/GANLSTMTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/bhpfelix/PyTorch-Time-Series-Classification-Benchmarks/blob/master/MaterialRecognitionModels.ipynb
import shutil, os, csv, itertools, glob

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn.metrics import confusion_matrix

import pandas as pd
import pickle as pk

cuda = torch.cuda.is_available()


In [2]:
## Multilayer LSTM based classifier taking in 200 dimensional fixed time series inputs
class LSTMClassifier(nn.Module):

    def __init__(self, in_dim, hidden_dim, num_layers, dropout, bidirectional, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        self.arch = 'lstm'
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_dir = 2 if bidirectional else 1
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
                input_size=in_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                dropout=dropout,
                bidirectional=bidirectional
            )

        self.hidden2label = nn.Sequential(
            nn.Linear(hidden_dim*self.num_dir, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, num_classes),
            nn.Sigmoid()
        )

        self.hidden = self.init_hidden()

    def init_hidden(self):
        if cuda:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, x): # x is (batch_size, 1, 200), permute to (200, batch_size, 1)
        x = x.permute(2, 0, 1)
        # See: https://discuss.pytorch.org/t/solved-why-we-need-to-detach-variable-which-contains-hidden-representation/1426/2
        lstm_out, (h, c) = self.lstm(x, self.init_hidden())
        y  = self.hidden2label(lstm_out[-1])
        return y
    
    def setBatchSize(self, batch_size = 1):
        self.batch_size = batch_size


In [3]:
def get_models(): # tuples of (batch_size, model)
    return [
         LSTMClassifier(
            in_dim=2,
            hidden_dim=120,
            num_layers=3,
            dropout=0.8,
            bidirectional=True,
            num_classes=1,#bce loss for discriminator
            batch_size=256
        )
    ]


In [4]:
model = get_models()[0]

In [5]:
model

LSTMClassifier(
  (lstm): LSTM(2, 120, num_layers=3, dropout=0.8, bidirectional=True)
  (hidden2label): Sequential(
    (0): Linear(in_features=240, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=120, out_features=120, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=120, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [6]:
x = torch.ones(256,2,200).float().cuda()
model = model.cuda()

In [7]:
model(x).shape


torch.Size([256, 1])

In [8]:
# Dummy Dataset for testing purpose only
class DummyDataset(Dataset):
    """Time Series dataset."""

    def __init__(self, numclasses=15):
        self.numclasses = numclasses

    def __len__(self):
        return 512

    def __getitem__(self, idx):
        data = None
        y = None
        if torch.rand(1)[0] > .5:
          data = torch.ones(2,200).float()
          y = float(1)
        else:
          data = torch.zeros(2,200).float()
          y = float(0)
          
        return data, y


In [9]:
data = DummyDataset(numclasses=2)
D = model.cuda()
train_loader = torch.utils.data.DataLoader(
    data,
    batch_size=256, shuffle=True)
D_optimizer = optim.Adam(D.parameters(), lr=.0001, betas=(0.5, 0.999))


In [10]:
bce = torch.nn.BCELoss()

In [11]:
for epoch in range(1):
  for x_, y_ in train_loader:
    y_ = y_.unsqueeze(dim = 1).float().cuda()
    x_ = x_.cuda()

    D_result = D(x_)
    #print(D_result)
    #break
    #print(D_result)
    D_train_loss = bce(D_result, y_)
    print('D_train_loss', D_train_loss)
    D_train_loss.backward()
    D_optimizer.step()



D_train_loss tensor(0.6944, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6936, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)


In [12]:
data1 = torch.ones(1,2,200).float()
y1 = torch.ones(1,1)
data0 = torch.zeros(1,2,200).float()
y0 = torch.zeros(1,1)
        

In [13]:
for x, y in train_loader:
  x = x.cuda()
  print(torch.sum(model(x)>.5))
  print(torch.sum(y))

tensor(65, device='cuda:0')
tensor(126., dtype=torch.float64)
tensor(62, device='cuda:0')
tensor(139., dtype=torch.float64)


In [14]:
# Dummy Dataset for testing purpose only
class DummyDataset2(Dataset):
    """Time Series dataset."""

    def __len__(self):
        return 512

    def __getitem__(self, idx):
        data = None
        y = None
        ind = [x for x in range(200)]
        ind = torch.tensor(ind).float()
        if torch.rand(1)[0] > .5:
          data = torch.ones(2,200).float()
          data[0,:] = torch.sin(ind) +torch.rand(1)[0]
          data[1,:] = torch.cos(ind) +torch.rand(1)[0]
          y = float(1)
        else:
          data = torch.ones(2,200).float()
          
          data[0,:] = torch.sin(ind*2.0) +torch.rand(1)[0] 
          data[1,:] = torch.cos(ind*2.0) +torch.rand(1)[0] 
          y = float(0)
          
        return data, y


In [15]:
data = DummyDataset2()
D = get_models()[0].cuda()
train_loader = torch.utils.data.DataLoader(
    data,
    batch_size=256, shuffle=True)
D_optimizer = optim.Adam(D.parameters(), lr=.0001, betas=(0.5, 0.999))


In [16]:
for epoch in range(1):
  for x_, y_ in train_loader:
    y_ = y_.unsqueeze(dim = 1).float().cuda()
    x_ = x_.cuda()

    D_result = D(x_)
    #print(D_result)
    #break
    #print(D_result)
    D_train_loss = bce(D_result, y_)
    print('D_train_loss', D_train_loss)
    D_train_loss.backward()
    D_optimizer.step()

D_train_loss tensor(0.6932, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6903, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)


In [17]:
#https://github.com/bhpfelix/PyTorch-Time-Series-Classification-Benchmarks/blob/master/MaterialRecognitionModels.ipynb

In [18]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=False)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=False)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(5, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=False)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

def resnet18(pretrained=False, progress=True, **kwargs):
    """ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

def resnext101_32x8d(pretrained=False, progress=True, **kwargs):
    """ResNeXt-101 32x8d model from
    Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

In [19]:
generator = resnet18(pretrained=False, progress=True).cuda()


In [20]:
a = torch.ones(256, 5, 32, 32).float().cuda()

In [21]:
generator(a)

tensor([[0.0177, 0.0293, 0.0113,  ..., 0.0056, 0.0023, 0.0123],
        [0.0177, 0.0293, 0.0113,  ..., 0.0056, 0.0023, 0.0123],
        [0.0177, 0.0293, 0.0113,  ..., 0.0056, 0.0023, 0.0123],
        ...,
        [0.0177, 0.0293, 0.0113,  ..., 0.0056, 0.0023, 0.0123],
        [0.0177, 0.0293, 0.0113,  ..., 0.0056, 0.0023, 0.0123],
        [0.0177, 0.0293, 0.0113,  ..., 0.0056, 0.0023, 0.0123]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [22]:
discriminator = get_models()[0].cuda()

In [23]:
discriminator

LSTMClassifier(
  (lstm): LSTM(2, 120, num_layers=3, dropout=0.8, bidirectional=True)
  (hidden2label): Sequential(
    (0): Linear(in_features=240, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=120, out_features=120, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=120, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [24]:
#x = torch.ones(256,2,200).float().cuda()


In [25]:
#discriminator(x).shape

In [26]:
#gt = torch.ones(256,1).cuda()

In [27]:
#loss = nn.BCELoss()


In [28]:
#output = discriminator(x)
#loss(output,gt)

In [29]:
#result = loss(output,gt)

In [30]:
#result.backward()


In [31]:
og = generator(a)

In [32]:
og.shape

torch.Size([256, 2000])

In [33]:
ogr = og.reshape(-1,2,1000)

In [34]:
od = discriminator(ogr)

In [35]:
od.shape

torch.Size([256, 1])

In [36]:
gtd = torch.ones(256,1).cuda()

In [37]:
loss = nn.BCELoss()

In [38]:
lossout = loss(od,gtd)

In [39]:
lossout.backward()

In [40]:
G =  resnet18(pretrained=False, progress=True).cuda()
D = get_models()[0].cuda() 
D_optimizer = optim.Adam(D.parameters(), lr=.0001, betas=(0.5, 0.999))
G_optimizer = optim.Adam(G.parameters(), lr=.0001, betas=(0.5, 0.999))
a = torch.ones(256, 5, 32, 32).float().cuda()
og = generator(a)
ogr = og.reshape(-1,2,1000)
od = discriminator(ogr)
gtd = torch.ones(256,1).cuda()
loss = nn.BCELoss()
lossout = loss(od,gtd)
lossout.backward()
G_optimizer.step()

In [41]:
G =  resnet18(pretrained=False, progress=True).cuda()
D = get_models()[0].cuda() 
D_optimizer = optim.Adam(D.parameters(), lr=.0001, betas=(0.5, 0.999))
G_optimizer = optim.Adam(G.parameters(), lr=.0001, betas=(0.5, 0.999))
a = torch.ones(256, 5, 32, 32).float().cuda()
og = generator(a)
ogr = og.reshape(-1,2,1000)
od = discriminator(ogr)
gtd = torch.ones(256,1).cuda()
loss = nn.BCELoss()
lossout = loss(od,gtd)
lossout.backward()
D_optimizer.step()

In [42]:
x = torch.ones(256,2,200).float().cuda()
out = discriminator(x).shape

In [54]:
x = torch.ones(1,2,1000).float().cuda()
discriminator.setBatchSize(batch_size = 1)
print(discriminator.init_hidden()[0].shape)
out = discriminator(x)
print(out)
print(out.shape)

torch.Size([6, 1, 120])
tensor([[0.5171]], device='cuda:0', grad_fn=<SigmoidBackward>)
torch.Size([1, 1])
